In [ ]:
from ngram_finder import NGramFinder
import csv
import nltk

nltk.download('stopwords')

## Change filepath and comment column index below

In [ ]:
filepath = '../data/2020-11-20 Stadteingang Elbbruecken_Kommentare.csv'

# for the elbbruecken file the comments are in the third column
comment_index = 3

In [ ]:
ngf = NGramFinder()

with open(filepath, 'r', encoding='utf8') as file:
    lines = list(csv.reader(file, delimiter=';'))
    comments = [line[comment_index] for line in lines]
    count = ngf.get_ngram_count(comments, 2, [])
    print(max(count.values()))
    for text, count in count.items():
        if count >= 5:

            print(text)